<a href="https://colab.research.google.com/github/Sabelz/Master_Thesis_Alexander/blob/main/GPs/PowerGPs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gaussian Processes on the Power Plant dataset

In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Master_Thesis_Alexander
!git config --global user.email "alexander.sabelstrom.1040@student.uu.se"
!git config --global user.name "Sabelz"

Mounted at /content/drive
/content/drive/MyDrive/Master_Thesis_Alexander


# Imports

In [8]:
import numpy as np
import matplotlib.pyplot as plt
import torch
!pip install gpytorch > \dev\null # Suppress prints
import gpytorch
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from torch.utils.data import TensorDataset, DataLoader

%matplotlib inline
%load_ext autoreload
%autoreload 2
%run "datasets/power.ipynb" # Run the Power notebook
%run "utils/models.ipynb" # Run the models notebook
%run "utils/functions.ipynb" # Run the functions notebook

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Master_Thesis_Alexander
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9568 entries, 0 to 9567
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   AT      9568 non-null   float64
 1   V       9568 non-null   float64
 2   AP      9568 non-null   float64
 3   RH      9568 non-null   float64
 4   PE      9568 non-null   float64
dtypes: float64(5)
memory usage: 373.9 KB
None

AT    False
V     False
AP    False
RH    False
PE    False
dtype: bool
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Master_Thesis_Alexander
----------------------------------------------------------
ALL MODELS: 
KIS

# Prepare The Data

In [6]:
powerPlant_data = df_PowerPlant # df_PowerPlant is defined in ../datasets/power.ipynb
# The last column is output(net hourly electrical energy output (EP)  of the plant)
x_power, y_power = powerPlant_data.iloc[:, :-1].to_numpy() , powerPlant_data.iloc[:, -1].to_numpy()


x_train_power, x_test_power, y_train_power, y_test_power = train_test_split(x_power, y_power, test_size=0.2, random_state=666)
# Transform into tensors
x_train_power, x_test_power, y_train_power, y_test_power = (
torch.from_numpy(x_train_power), torch.from_numpy(x_test_power),
torch.from_numpy(y_train_power), torch.from_numpy(y_test_power))

if torch.cuda.is_available():
    x_train_power, y_train_power, x_test_power, y_test_power = (
    x_train_power.cuda(), y_train_power.cuda(), x_test_power.cuda(), y_test_power.cuda())

# Train data into batches
train_dataset_power = TensorDataset(x_train_power, y_train_power)
train_loader_power = DataLoader(train_dataset_power, batch_size=500, shuffle=True)

# Test data into batches
test_dataset_power = TensorDataset(x_test_power, y_test_power)
test_loader_power = DataLoader(test_dataset_power, batch_size=500, shuffle=False)
